# 🤖 OpenVLA Action Tokens 실습 (Colab + T4 GPU)

**목표:**
1. 실제 OpenVLA 7B 모델 로드
2. 이미지 + 명령어 → Action tokens 추출
3. Token → Normalized action → Real action 변환 전체 pipeline 이해

**요구사항:**
- GPU: T4 (16GB) ✅
- Runtime: Python 3
- GPU 활성화: Runtime → Change runtime type → T4 GPU

## 📦 Step 1: 환경 설정

In [ ]:
# GPU 확인
!nvidia-smi

In [ ]:
# 필요한 라이브러리 설치
print("📦 Installing required libraries...")
!pip install -q transformers torch pillow huggingface_hub
print("✅ Installation complete!")

In [ ]:
# Import libraries
import torch
import numpy as np
from PIL import Image
import json

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

## 🤖 Step 2: OpenVLA 모델 로드

**주의:** 첫 실행 시 ~14GB 다운로드 (5-10분 소요)

In [ ]:
from transformers import AutoModelForVision2Seq, AutoProcessor

MODEL_ID = "openvla/openvla-7b"

print(f"🤖 Loading OpenVLA model: {MODEL_ID}")
print("   This may take 5-10 minutes on first run...\n")

# Processor 로드
processor = AutoProcessor.from_pretrained(
    MODEL_ID,
    trust_remote_code=True
)
print("✅ Processor loaded")

# 모델 로드 (bfloat16으로 VRAM 절약)
vla = AutoModelForVision2Seq.from_pretrained(
    MODEL_ID,
    torch_dtype=torch.bfloat16,
    low_cpu_mem_usage=True,
    trust_remote_code=True
).to("cuda")

print("\n✅ Model loaded successfully!")
print(f"   Device: {vla.device}")
print(f"   Dtype: {vla.dtype}")

## 🖼️ Step 3: 테스트 이미지 준비

In [ ]:
# 방법 1: 더미 이미지 생성
dummy_image = Image.new('RGB', (224, 224), color=(70, 130, 180))  # Steel blue
dummy_image.save('/tmp/test_image.png')

# 이미지 확인
display(dummy_image)
print("✅ Test image created: /tmp/test_image.png")

# 방법 2: 실제 이미지 업로드하려면 아래 주석 해제
# from google.colab import files
# uploaded = files.upload()
# image_path = list(uploaded.keys())[0]
# test_image = Image.open(image_path)

## 🎯 Step 4: Action Prediction (전체 Pipeline)

In [ ]:
# 입력 설정
image_path = '/tmp/test_image.png'
instruction = "pick up the blue block"
unnorm_key = "bridge_orig"  # Bridge dataset statistics 사용

print("=" * 70)
print("🎯 OpenVLA Action Prediction")
print("=" * 70)
print(f"\n[입력]")
print(f"  Image: {image_path}")
print(f"  Instruction: {instruction}")
print(f"  Dataset: {unnorm_key}")

# 이미지 로드
image = Image.open(image_path).convert("RGB")

# Prompt 생성 (OpenVLA 형식)
prompt = f"In: What action should the robot take to {instruction.lower()}?\nOut:"
print(f"\n  Prompt: {prompt}")

# Processor로 입력 처리
inputs = processor(prompt, image).to(vla.device, dtype=vla.dtype)

# Action 예측 (간단한 방법)
print(f"\n[Inference] Predicting action...")
with torch.no_grad():
    action = vla.predict_action(
        **inputs,
        unnorm_key=unnorm_key,
        do_sample=False  # Greedy decoding
    )

print(f"\n✅ Prediction complete!")
print(f"\n[출력] Real Robot Action:")
print(f"  {action}")

## 🔍 Step 5: Action Token 추출 및 분석 (핵심!)

In [ ]:
print("=" * 70)
print("🔍 Action Token 분석")
print("=" * 70)

# Step 1: 모델이 생성한 token sequence 추출
print("\n[Step 1] Generating action tokens...")
action_dim = 7  # Bridge dataset: [6 EEF + 1 gripper]

with torch.no_grad():
    generated_ids = vla.generate(
        **inputs,
        max_new_tokens=action_dim,
        do_sample=False
    )

# 마지막 7개 token 추출 (openvla.py:90)
action_token_ids = generated_ids[0, -action_dim:].cpu().numpy()

print(f"  Generated token IDs: {action_token_ids}")
print(f"  Shape: {action_token_ids.shape}")
print(f"  Vocab size: {vla.config.vocab_size}")

# Step 2: Token → Normalized action [-1, 1]
print(f"\n[Step 2] Token IDs → Normalized actions")
normalized_actions = vla.action_tokenizer.decode_token_ids_to_actions(
    action_token_ids
)

print(f"  Normalized actions: {normalized_actions}")
print(f"  Range: [{normalized_actions.min():.3f}, {normalized_actions.max():.3f}]")

# Step 3: Dataset statistics 로드
print(f"\n[Step 3] Loading dataset statistics (unnorm_key={unnorm_key})")
action_stats = vla.get_action_stats(unnorm_key)

q01 = np.array(action_stats["q01"])
q99 = np.array(action_stats["q99"])
mask = action_stats.get("mask", np.ones_like(q01, dtype=bool))

print(f"  q01 (1% quantile): {q01}")
print(f"  q99 (99% quantile): {q99}")
print(f"  mask: {mask}")

# Step 4: Un-normalization (openvla.py:97-101)
print(f"\n[Step 4] Un-normalization → Real robot commands")
real_actions = 0.5 * (normalized_actions + 1) * (q99 - q01) + q01
real_actions = np.where(mask, real_actions, normalized_actions)

print(f"  Real actions: {real_actions}")

# 검증: predict_action() 결과와 비교
print(f"\n[Verification]")
print(f"  predict_action() result: {action}")
print(f"  Manual calculation:      {real_actions}")
print(f"  Difference: {np.abs(action - real_actions)}")
print(f"  ✅ Match: {np.allclose(action, real_actions)}")

## 📊 Step 6: 상세 분석 및 시각화

In [ ]:
import pandas as pd

# 차원별 분석 테이블
dim_names = [
    "X-axis delta (m)",
    "Y-axis delta (m)",
    "Z-axis delta (m)",
    "Roll delta (rad)",
    "Pitch delta (rad)",
    "Yaw delta (rad)",
    "Gripper (0/1)"
]

# DataFrame 생성
df = pd.DataFrame({
    "Dimension": dim_names,
    "Token ID": action_token_ids,
    "Bin Index": action_token_ids - (vla.config.vocab_size - 256),
    "Normalized": normalized_actions,
    "q01": q01,
    "q99": q99,
    "Real Action": real_actions,
    "Mask": mask
})

print("\n" + "=" * 70)
print("📊 Action Token 상세 분석")
print("=" * 70)
print(df.to_string(index=False))

# 움직임 해석
print("\n" + "=" * 70)
print("🤖 로봇 동작 해석")
print("=" * 70)

for i in range(6):
    if abs(real_actions[i]) > 0.001:
        print(f"  {dim_names[i]:25s}: {real_actions[i]:+.6f}")
        if i < 3:  # XYZ
            print(f"    → {abs(real_actions[i]*100):.2f}cm 이동")
        else:  # Rotation
            print(f"    → {abs(np.degrees(real_actions[i])):.2f}° 회전")

gripper_status = "OPEN" if real_actions[6] > 0.5 else "CLOSE"
print(f"\n  Gripper: {gripper_status} ({real_actions[6]:.2f})")

## 🎮 Step 7: 여러 명령어로 실험

In [ ]:
# 여러 명령어 테스트
instructions = [
    "pick up the blue block",
    "move forward",
    "grasp the cup",
    "open the gripper",
    "lift the object"
]

print("=" * 70)
print("🎮 Multiple Instructions Test")
print("=" * 70)

results = []

for inst in instructions:
    print(f"\n📝 Instruction: {inst}")
    
    # Prompt 생성
    prompt = f"In: What action should the robot take to {inst.lower()}?\nOut:"
    inputs = processor(prompt, image).to(vla.device, dtype=vla.dtype)
    
    # 예측
    with torch.no_grad():
        action = vla.predict_action(
            **inputs,
            unnorm_key="bridge_orig",
            do_sample=False
        )
    
    print(f"   Action: {action}")
    results.append({"instruction": inst, "action": action})

# 결과 비교
print("\n" + "=" * 70)
print("📊 Results Comparison")
print("=" * 70)
for r in results:
    print(f"\n{r['instruction']:30s}")
    print(f"  XYZ: [{r['action'][0]:+.4f}, {r['action'][1]:+.4f}, {r['action'][2]:+.4f}]")
    print(f"  Gripper: {r['action'][6]:.2f}")

## 💾 Step 8: Dataset Statistics 저장

In [ ]:
# 실제 dataset statistics 저장
all_stats = vla.norm_stats

# JSON으로 저장
with open('/tmp/real_dataset_statistics.json', 'w') as f:
    # numpy array를 list로 변환
    def convert_to_serializable(obj):
        if isinstance(obj, dict):
            return {k: convert_to_serializable(v) for k, v in obj.items()}
        elif isinstance(obj, (list, tuple)):
            return [convert_to_serializable(item) for item in obj]
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        elif isinstance(obj, (np.integer, np.floating)):
            return obj.item()
        else:
            return obj
    
    serializable_stats = convert_to_serializable(all_stats)
    json.dump(serializable_stats, f, indent=2)

print("✅ Dataset statistics saved to: /tmp/real_dataset_statistics.json")
print(f"\nAvailable datasets: {list(all_stats.keys())}")

# 다운로드
from google.colab import files
files.download('/tmp/real_dataset_statistics.json')

## 📝 Step 9: 학습 요약

In [ ]:
print("=" * 70)
print("📚 학습 요약")
print("=" * 70)

print("""
✅ 완료한 내용:

1. OpenVLA 7B 모델 로드 (T4 GPU)
   - Processor 초기화
   - Model 초기화 (bfloat16)

2. Action Prediction Pipeline:
   - Image + Instruction → Prompt 생성
   - Model.generate() → Token IDs 생성
   - Token IDs → Normalized actions [-1, 1]
   - Un-normalization → Real robot commands

3. 핵심 개념:
   - Action Tokenization: 연속값 → 256 bins → Token IDs
   - BOUNDS_Q99 Normalization: (x - q01) / (q99 - q01)
   - Un-normalization: 0.5 * (norm + 1) * (q99 - q01) + q01
   - Mask: Gripper는 정규화 안 함

4. 실전 경험:
   - 실제 모델로 token 추출
   - Dataset statistics 활용
   - 여러 명령어로 실험

📌 다음 단계:
   - RLDS 데이터 포맷 이해
   - Fine-tuning 실습
   - 실제 로봇 제어 (Control Frequency)
""")

print("\n" + "=" * 70)
print("🎉 실습 완료!")
print("=" * 70)

## 💡 추가 실험 (선택)

### 실제 이미지 업로드해서 테스트:

```python
from google.colab import files
uploaded = files.upload()
image_path = list(uploaded.keys())[0]

image = Image.open(image_path).convert("RGB")
instruction = "your custom instruction"

# ... (위 코드 실행)
```

### 다른 Dataset 사용:

```python
# bridge_orig 대신 다른 dataset
unnorm_key = "fractal20220817_data"
action = vla.predict_action(**inputs, unnorm_key=unnorm_key)
```